### Redes Neuronales con Capas Ocultas.

Situación: Quieres predecir si un estudiante aprueba (1) o no (0) basado en horas de estudio y horas de sueño.

Usarás una red neuronal con una capa oculta.

In [5]:
import numpy as np

# Datos: [horas_estudio, horas_sueño]
X = np.array([[2, 8], [3, 7], [4, 6], [5, 5], [6, 4]])
y = np.array([0, 0, 1, 1, 1]) # 0 = No aprueba, 1 = Aprueba

# Hiperparámetros.
tasa_aprendizaje = 0.1
epocas = 10000

# Inicializar pesos y sesgos (aleatorios)
# Capa oculta: 2 neuronas
pesos_entrada_oculta = np.random.randn(2, 2) # 2 entradas, 2 neuronas ocultas.
bias_ocultas = np.random.randn(2)

# Capa salida: 1 neurona.
pesos_oculta_salida = np.random.randn(2, 1)
bias_salida = np.random.randn(1)

# Función de activación sigmoide
def sigmoide(x):
    return 1 / (1 + np.exp(-x))

# Derivada de la sigmoide (para backpropagation)
def derivada_sigmoide(x):
    return x * (1 - x)

# Entrenamiento
for epoch in range(epocas):
    # Forward propagation
    # Capa oculta
    z_oculta = np.dot(X, pesos_entrada_oculta) + bias_ocultas
    activacion_oculta = sigmoide(z_oculta)

    # Capa salida
    z_salida = np.dot(activacion_oculta, pesos_oculta_salida) + bias_salida
    activacion_salida = sigmoide(z_salida)

    # Backpropagation
    error = activacion_salida - y.reshape(-1, 1)

    # Gradientes capa salida
    delta_salida = error * derivada_sigmoide(activacion_salida)

    # Gradientes capa oculta
    error_oculta = delta_salida.dot(pesos_oculta_salida.T)
    delta_oculta = error_oculta * derivada_sigmoide(activacion_oculta)

    # Actualizar pesos y sesgos
    pesos_oculta_salida -= tasa_aprendizaje * activacion_oculta.T.dot(delta_salida)
    bias_salida -= tasa_aprendizaje * np.sum(delta_salida, axis = 0)

    pesos_entrada_oculta -= tasa_aprendizaje * X.T.dot(delta_oculta)
    bias_ocultas -= tasa_aprendizaje * np.sum(delta_oculta, axis = 0)

# Predicción final
predicciones = sigmoide(np.dot(sigmoide(np.dot(X, pesos_entrada_oculta) + bias_ocultas), pesos_oculta_salida) + bias_salida)
print("Predicciones finales:", predicciones.flatten())

Predicciones finales: [0.02560257 0.0328198  0.97466088 0.98386983 0.98379552]


* Forward propagation: Calcula las actividades capa por capa.
* Backpropagation: Ajusta los pesos basado en el error.
* Derivada de la sigmoide: Crucial para calcular gradientes.

Modifica el código para tener:

1. 3 neuronas en la capa oculta.
2. Función de activación ReLU en la capa oculta (y su derivada).

In [5]:
import numpy as np

# Datos
X = np.array([[2, 8], [3, 7], [4, 6], [5, 5], [6, 4]])
y = np.array([0, 0, 1, 1, 1])

# Hiperparametros
tasa_aprendizaje = 0.1
epocas = 10000

# Inicializar pesos y sesgos (aleatorios)
# Capa oculta: 3 neuronas
pesos_entrada_oculta = np.random.randn(2, 3)
bias_ocultas = np.random.randn(3)

# Capa salida
pesos_oculta_salida = np.random.randn(3, 1)
bias_salida = np.random.randn(1)

# Activación ReLU
def relu(x):
    return np.maximum(0, x)

# Derivada ReLU
def derivada_relu(x):
    return np.where(x > 0, 1, 0)

# Entrenamiento
for epoch in range(epocas):
    # Forward propagation
    # Capa oculta
    z_oculta = np.dot(X, pesos_entrada_oculta) + bias_ocultas
    activacion_oculta = relu(z_oculta)

    # Capa salida
    z_salida = np.dot(activacion_oculta, pesos_oculta_salida) + bias_salida
    activacion_salida = relu(z_salida)

    # Backpropagation
    error = activacion_salida - y.reshape(-1, 1)

    # Gradientes capa salida
    delta_salida = error * derivada_relu(activacion_salida)

    # Gradientes capa oculta
    error_oculta = delta_salida.dot(pesos_oculta_salida.T)
    delta_oculta = error_oculta * derivada_relu(activacion_oculta)

    # Actualizar pesos y sesgos
    pesos_oculta_salida -= tasa_aprendizaje * activacion_oculta.T.dot(delta_salida)
    bias_salida -= tasa_aprendizaje * np.sum(delta_salida, axis = 0)

# Predicción final
predicciones = relu(np.dot(relu(np.dot(X, pesos_entrada_oculta) + bias_ocultas), pesos_oculta_salida) + bias_salida)
print("Predicciones finales:", predicciones.flatten())

Predicciones finales: [0. 0. 0. 0. 0.]


La red "se murió": todas las salidas quedaron en cero.

Esto pasa porque:

1. ReLU en la salida no es buena idea en clasificación binaria -> porque nunca empuja las salidas hacia "probabilidad". Se queda en valores arbitrarios, y si se saturan en 0, no hay gradiente.
2. El problema del "dying ReLU": cuando todos los **z** son negativos en alguna capa, la ReLU los manda a 0, la derivada es 0 -> y ya no aprende más.

En este caso, la arquitectura estándar para el problema debería ser:

* Capa oculta: ReLU.
* Capa de salida: Sigmoide (para binario) o Softmax (si fueran varias clases).

---

### Optimización de Hiperparámetros con GridSearchCV

Situación: Quieres encontrar los mejores hiperparámetros para un RandomForest que predice cáncer de mama.

In [7]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV

# Cargar datos
datos = load_breast_cancer()
X = datos.data
y = datos.target

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Definir el modelo
modelo = RandomForestClassifier(random_state = 42)

# Definir la grilla de hiperparámetros
param_grid = {
    'n_estimators': [50, 100, 200], # Número de árboles
    'max_depth': [None, 10, 20], # Profundidad máxima
    'min_samples_split': [2, 5, 10] # Mínimo muestras para dividir un nodo
}

# Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator = modelo,
    param_grid = param_grid,
    cv = 5, # 5-fold cross-validation
    scoring = 'accuracy',
    n_jobs = -1 # Usar todos los cores disponibles
)

# Ejecutar busqueda
grid_search.fit(X_train, y_train)

# Mejores parámetros
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor accuracy (validación):", grid_search.best_score_)

# Evaluar un test
mejor_modelo = grid_search.best_estimator_
y_pred = mejor_modelo.predict(X_test)
print("Accuracy en test:", accuracy_score(y_test, y_pred))

Mejores parámetros: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Mejor accuracy (validación): 0.9626373626373625
Accuracy en test: 0.9649122807017544


* GridSearchCV: Prueba todas las combinaciones de hiperparámetros.
* cv = 5: Valida cada combinación con 5-fold cross-validation.
* best_estimator_: El modelo ya entrenado con los mejores parámetros.

Ejercicio:

Usa GridSearchCV para optimizar:

* max_depth y min_samples_split en un DecisionTreeClassifier.
* Usa el dataset de iris (load_iris)